In [1]:
import pandas as pd
import sys, getopt
import numpy as np
import pprint
import os
from datetime import datetime
import glob
import pandas as pd
import emgimporter
import pickle
import time

def augmentData(x,y,nb_roll,steps):
    assert len(x.shape) > 1
    x_cp = x
    y_cp = y
    for i in range(-nb_roll,nb_roll,steps):
        x_cp = np.vstack((x_cp,np.roll(x,i,axis=1)))
        y_cp = np.vstack((y_cp,y))

    return x_cp,y_cp


dic = { 1: 'Tasse aufnehmen',
        2: 'Tasse halten',
        3: 'Tasse abstellen',
        4: 'Tasse hoch&runter',
        8: 'Ruhe (Supination)',
        9: 'Ruhe (Pronation)'
      }
its = ['cupv1','cupv2','kettlev1','data','loosev1','jascha','markus','Sirius']

data_arch = {   0: (7,8,9),
                1: (2,4),
             }

feed_dic = emgimporter.import_folder(its,dic,path=['emg_data','ipa_emg'])

x,y = emgimporter.prep_data(feed_dic,data_arch)

y_cut_train, y_cut_test, x_cut_train, x_cut_test = emgimporter.split_data(x,y,splitratio=0.05,shuffle=True)

x,y = augmentData(x_cut_train,y_cut_train,2,1)


Using TensorFlow backend.


In [13]:
# Prep Data for SVM classifier

# Train data
svm_y_train_ = y
svm_y_train_[:,0] *= 0
svm_y_train = np.sum(svm_y_train_,axis=1)

# Test data
svm_y_test_ = y_cut_test
svm_y_test_[:,0] *= 0
svm_y_test = np.sum(svm_y_test_,axis=1)


In [31]:
# Train SVM Classifier
# Default kernel: RBF w/ γ = 1/n_features

from sklearn.svm import SVC
clf = SVC(verbose=True)
clf.fit(x/255,svm_y_train)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [15]:
from sklearn.metrics import accuracy_score

y_true = k2_test
y_pred_svm = clf.predict(x_cut_test)
svm_acc = accuracy_score(y_true,y_pred_svm)
print("SVM Score: ", svm_acc)

SVM Score:  0.737177445046


In [18]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=2, weights='distance')
neigh.fit(x, svm_y_train) 

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='distance')

In [19]:
y_pred_knn = neigh.predict(x_cut_test)
knn_acc = accuracy_score(y_true,y_pred_knn)
print("KNN Score: ", knn_acc)

KNN Score:  0.957948391207


In [29]:
kernelDict = {'rbf':[{'gamma':2**f} for f in range(-15,3,1)],'linear':{},'polynomial':{'degree':2,'coef0':1}, 'sigmoid':{'coef0':1}}

In [28]:
print([r for r in range(0,2,1)])

[0, 1]


In [32]:
a = 2**np.linspace(-15,3,20)
b = 2**np.linspace(-5,15,20)

In [33]:
a

array([  3.05175781e-05,   5.88486327e-05,   1.13480878e-04,
         2.18831078e-04,   4.21983346e-04,   8.13732428e-04,
         1.56916255e-03,   3.02589774e-03,   5.83499594e-03,
         1.12519261e-02,   2.16976741e-02,   4.18407532e-02,
         8.06836999e-02,   1.55586574e-01,   3.00025680e-01,
         5.78555119e-01,   1.11565792e+00,   2.15138117e+00,
         4.14862018e+00,   8.00000000e+00])

In [34]:
a.shape

(20,)

In [35]:
cc = np.vstack((a,b))

In [57]:
cc = np.array(range(0,9))

In [62]:
cc =cc.reshape((3,3))

In [66]:
kk = np.pad(cc,((0,0),(2,2)),'wrap')

In [74]:
kk.shape

(3, 7)

In [68]:
kk

array([[1, 2, 0, 1, 2, 0, 1],
       [4, 5, 3, 4, 5, 3, 4],
       [7, 8, 6, 7, 8, 6, 7]])

In [77]:
a= [(0,0) for f in range(0,9)]

In [79]:
axis = 2
a[axis] = (2,2)

In [80]:
a

[(0, 0), (0, 0), (2, 2), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0), (0, 0)]

In [81]:
len(kk.shape)

2

In [82]:
def padStuff(matrix,expsize=(2,2),axis=0,mode='wrap'):
    dim = len(matrix.shape)
    tup1 = [(0,0) for _ in range(0,dim)]
    tup1[axis] = expsize
    return np.pad(matrix,tup1,mode)

In [85]:
padStuff(kk,axis=1)

array([[0, 1, 1, 2, 0, 1, 2, 0, 1, 1, 2],
       [3, 4, 4, 5, 3, 4, 5, 3, 4, 4, 5],
       [6, 7, 7, 8, 6, 7, 8, 6, 7, 7, 8]])

In [86]:
kk

array([[1, 2, 0, 1, 2, 0, 1],
       [4, 5, 3, 4, 5, 3, 4],
       [7, 8, 6, 7, 8, 6, 7]])